In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import RocCurveDisplay
from sklearn.metrics import accuracy_score, classification_report
pd.set_option('display.max_columns',None)
import joblib

In [2]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D90.parquet')
test_day.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_90,Smoothed_Close_90,down_days,up_days,RSI,low_14,high_14,k_percent_14,r_percent_14,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
27039,2025-01-21,54.290001,54.320000,53.250000,54.020000,54.020000,28798700,VALE3.SA,-0.270000,54.608148,NaN,54.020000,NaN,NaN,NaN,0.270000,0.000000,44.550340,50.869999,55.099998,88.372335,-11.627665,-1.093392,-1.038177,-0.248217,3562882492
27040,2025-01-22,54.279999,54.290001,52.419998,52.660000,52.660000,34645400,VALE3.SA,-1.360001,54.491259,NaN,52.660000,NaN,NaN,NaN,1.360001,0.000000,42.417665,50.869999,54.490002,100.034744,0.034744,-1.078050,-1.046151,-0.252501,3528237092
27041,2025-01-23,52.950001,52.990002,51.840000,52.320000,52.320000,25097300,VALE3.SA,-0.340000,54.360984,NaN,52.320000,NaN,NaN,NaN,0.340000,0.000000,41.903787,50.869999,54.490002,96.435974,-3.564026,-1.064136,-1.049748,-0.256425,3503139792
27042,2025-01-24,52.599998,53.189999,52.599998,53.029999,53.029999,20520600,VALE3.SA,0.709999,54.281125,NaN,53.029999,NaN,NaN,NaN,0.000000,0.709999,43.371760,50.869999,54.490002,94.229923,-5.770077,-1.047479,-1.049295,-0.259676,3482619192
27043,2025-01-27,52.889999,53.009998,52.570000,52.910000,52.910000,1252600,VALE3.SA,-0.119999,54.198857,NaN,52.910000,NaN,NaN,NaN,0.119999,0.000000,43.182790,50.869999,54.490002,91.957342,-8.042658,-1.029054,-1.045246,-0.263257,3481366592


In [2]:
test_day = pd.read_parquet(f'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')
test_day.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
27011,2025-01-13,52.000000,52.310001,50.869999,51.509998,51.509998,30845300,VALE3.SA,-0.010002,51.887704,NaN,51.509998,NaN,NaN,NaN,0.010002,0.000000,22.087279,50.869999,55.400002,22.465881,-77.534119,-1.594897,-1.362351,-0.056011,124437362
27012,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,51.876392,NaN,51.849998,NaN,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,22.216176,-77.783824,-1.614043,-1.412690,-0.054067,103058362
27013,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,52.093474,NaN,52.599998,NaN,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,27.008267,-72.991733,-1.593333,-1.448818,-0.047025,138196062
27014,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,52.266431,NaN,52.669998,NaN,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,30.826305,-69.173695,-1.545152,-1.468085,-0.036330,358458862
27015,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,52.933502,NaN,54.490002,NaN,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,47.766288,-52.233712,-1.436582,-1.461784,-0.014894,413117562


In [3]:
test_day['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [3]:
test_day_AMBEV = test_day[test_day['Ticker'] == 'ABEV3.SA'].reset_index(drop=True)
test_day_AMBEV['Prediction'] = np.NaN

test_day_AMBEV.tail()

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_90,Smoothed_Close_90,down_days,up_days,RSI,low_14,high_14,k_percent_14,r_percent_14,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5746,2025-01-21,10.99,11.26,10.99,11.25,11.25,26293900,ABEV3.SA,0.20,11.921847,NaN,11.25,NaN,NaN,NaN,0.00,0.20,41.458624,10.77,11.89,102.843491,2.843491,-0.246361,-0.180701,-0.110465,-3688235895
5747,2025-01-22,11.18,11.22,10.94,11.02,11.02,44993500,ABEV3.SA,-0.23,11.867737,NaN,11.02,NaN,NaN,NaN,0.23,0.00,40.026011,10.77,11.58,135.523072,35.523072,-0.257227,-0.196006,-0.117102,-3733229395
5748,2025-01-23,11.02,11.16,10.98,11.05,11.05,35708100,ABEV3.SA,0.03,11.818672,NaN,11.05,NaN,NaN,NaN,0.00,0.03,40.301725,10.77,11.58,129.465761,29.465761,-0.266724,-0.210150,-0.123874,-3768937495
5749,2025-01-24,11.05,11.13,10.87,10.87,10.87,44514500,ABEV3.SA,-0.18,11.761752,NaN,10.87,NaN,NaN,NaN,0.18,0.00,39.193919,10.77,11.58,122.438552,22.438552,-0.275665,-0.223253,-0.130029,-3813451995
5750,2025-01-27,10.87,10.92,10.84,10.91,10.91,810800,ABEV3.SA,0.04,11.710647,NaN,10.91,NaN,NaN,NaN,0.00,0.04,39.572149,10.77,11.53,123.769393,23.769393,-0.283605,-0.235323,-0.135958,-3814262795


In [4]:
test_day_AMBEV['Date'] = pd.to_datetime(test_day_AMBEV['Date'])

# Definir a data inicial para filtrar
inicio = '2020-01-01'  # Data inicial

# Filtrar as linhas onde a data é maior ou igual a 'inicio'
test_day_AMBEV = test_day_AMBEV[test_day_AMBEV['Date'] >= inicio]

test_day_AMBEV.tail(15)


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_90,Smoothed_Close_90,down_days,up_days,RSI,low_14,high_14,k_percent_14,r_percent_14,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5736,2025-01-07,11.30,11.58,11.27,11.54,11.54,34905200,ABEV3.SA,0.25,12.527109,NaN,11.54,NaN,NaN,NaN,0.00,0.25,41.798448,11.21,13.42,59.597677,-40.402323,-0.060966,0.008806,-0.053187,-3346968695
5737,2025-01-08,11.48,11.53,11.28,11.42,11.42,39248800,ABEV3.SA,-0.12,12.460682,NaN,11.42,NaN,NaN,NaN,0.12,0.00,41.075465,11.21,13.42,56.591952,-43.408048,-0.084448,-0.009845,-0.059782,-3386217495
5738,2025-01-09,11.43,11.51,11.30,11.40,11.40,24470100,ABEV3.SA,-0.02,12.397041,NaN,11.40,NaN,NaN,NaN,0.02,0.00,40.954483,11.21,13.19,59.951590,-40.048410,-0.106961,-0.029268,-0.065375,-3410687595
5739,2025-01-10,11.40,11.43,11.08,11.16,11.16,31277300,ABEV3.SA,-0.24,12.322819,NaN,11.16,NaN,NaN,NaN,0.24,0.00,39.523106,11.08,13.08,62.140942,-37.859058,-0.129300,-0.049275,-0.072254,-3441964895
5740,2025-01-13,11.18,11.31,11.12,11.24,11.24,28945300,ABEV3.SA,0.08,12.257850,NaN,11.24,NaN,NaN,NaN,0.00,0.08,40.236550,11.08,12.91,64.363374,-35.636626,-0.150512,-0.069522,-0.078259,-3470910195
5741,2025-01-14,11.21,11.37,11.15,11.21,11.21,42625300,ABEV3.SA,-0.03,12.194979,NaN,11.21,NaN,NaN,NaN,0.03,0.00,40.054979,11.08,12.34,88.490359,-11.509641,-0.170431,-0.089704,-0.083365,-3513535495
5742,2025-01-15,11.30,11.43,11.16,11.39,11.39,45380700,ABEV3.SA,0.18,12.146680,NaN,11.39,NaN,NaN,NaN,0.00,0.18,41.673223,11.08,12.34,84.657129,-15.342871,-0.187948,-0.109353,-0.088130,-3558916195
5743,2025-01-16,11.42,11.42,11.04,11.08,11.08,29298500,ABEV3.SA,-0.31,12.082679,NaN,11.08,NaN,NaN,NaN,0.31,0.00,39.778185,11.04,12.24,86.889946,-13.110054,-0.204636,-0.128409,-0.094278,-3588214695
5744,2025-01-17,10.84,11.09,10.77,11.09,11.09,51433600,ABEV3.SA,0.01,12.023118,NaN,11.09,NaN,NaN,NaN,0.00,0.01,39.868571,10.77,12.19,88.247797,-11.752203,-0.220129,-0.146753,-0.098579,-3639648295
5745,2025-01-20,11.05,11.11,10.79,11.05,11.05,22293700,ABEV3.SA,-0.04,11.964731,NaN,11.05,NaN,NaN,NaN,0.04,0.00,39.624813,10.77,12.16,85.951898,-14.048102,-0.234417,-0.164286,-0.104375,-3661941995


##### Carregar modelo aprendido geral

In [15]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/random_forest_model_D10_0.99.pkl')

#### Carregar modelo aprendido especifico.

#### Ambev

In [17]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Ambev.pkl')

In [5]:
# Carregar o modelo salvo
rand_frst_clf_Ambev = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referência D_90 - 0.06/random_forest_model_Ambev.pkl')

#### Itau

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Itau = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Itau.pkl')

#### Magalu

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Magalu = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Magalu.pkl')

#### Petrobrás

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Petro = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Petro.pkl')

#### Vale

In [ ]:
# Carregar o modelo salvo
rand_frst_clf_Vale = joblib.load('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Vale.pkl')

In [6]:
import pandas as pd
import numpy as np
import joblib

# Carregar o DataFrame
test_day = pd.read_parquet('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/price_data_D10.parquet')

# Converter a coluna de datas para datetime
test_day['Date'] = pd.to_datetime(test_day['Date'])

# Definir a data inicial para filtrar
inicio = '2024-12-05'

# Lista de tickers e modelos correspondentes
tickers_models = {
    'ABEV3.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Ambev.pkl',
    'ITUB4.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Itau.pkl',
    'MGLU3.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Magalu.pkl',
    'PETR4.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Petro.pkl',
    'VALE3.SA': 'C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Modelos referências D_10 - 0.3/random_forest_model_Vale.pkl',
}

# Resultado final armazenado em um DataFrame
final_predictions = []

# Loop para processar cada Ticker
for ticker, model_path in tickers_models.items():
    print(f"Processando {ticker}...")
    
    # Filtrar o DataFrame para o Ticker atual
    test_day_ticker = test_day[test_day['Ticker'] == ticker].reset_index(drop=True)
    
    # Filtrar as datas a partir da data inicial
    test_day_ticker = test_day_ticker[test_day_ticker['Date'] >= inicio]
    
    # Verificar se há dados suficientes
    if test_day_ticker.empty:
        print(f"  Nenhum dado disponível para {ticker} após {inicio}.")
        continue
    
    # Selecionar as features para o modelo
    Features = test_day_ticker[['RSI', 'k_percent', 'r_percent', 'Price_Rate_Of_Change', 'MACD', 'MACD_EMA', 'On Balance Volume']].to_numpy()
    
    # Carregar o modelo correspondente
    model = joblib.load(model_path)
    
    # Fazer previsões
    predictions = model.predict(Features)
    
    # Adicionar as previsões ao DataFrame
    test_day_ticker['Prediction'] = predictions
    
    # Armazenar os resultados no conjunto final
    final_predictions.append(test_day_ticker)

# Concatenar todos os resultados em um único DataFrame
final_result_df = pd.concat(final_predictions, ignore_index=True)

final_result_df.tail()



Processando ABEV3.SA...
Processando ITUB4.SA...


c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Processando MGLU3.SA...
Processando PETR4.SA...


c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Processando VALE3.SA...


c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_10,Smoothed_Close_10,down_days,up_days,RSI,low_14,high_14,k_percent,r_percent,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
135,2025-01-13,52.000000,52.310001,50.869999,51.509998,51.509998,30845300,VALE3.SA,-0.010002,51.887704,1.0,51.509998,NaN,NaN,NaN,0.010002,0.000000,22.087279,50.869999,55.400002,22.465881,-77.534119,-1.594897,-1.362351,-0.056011,124437362
136,2025-01-14,51.950001,52.430000,51.630001,51.849998,51.849998,21379000,VALE3.SA,0.340000,51.876392,1.0,51.849998,NaN,NaN,NaN,0.000000,0.340000,30.128698,50.869999,55.400002,22.216176,-77.783824,-1.614043,-1.412690,-0.054067,103058362
137,2025-01-15,52.139999,52.680000,51.630001,52.599998,52.599998,35137700,VALE3.SA,0.750000,52.093474,1.0,52.599998,NaN,NaN,NaN,0.000000,0.750000,44.665010,50.869999,55.400002,27.008267,-72.991733,-1.593333,-1.448818,-0.047025,138196062
138,2025-01-16,52.290001,53.270000,52.000000,52.669998,52.669998,220262800,VALE3.SA,0.070000,52.266431,1.0,52.669998,NaN,NaN,NaN,0.000000,0.070000,45.877603,50.869999,55.400002,30.826305,-69.173695,-1.545152,-1.468085,-0.036330,358458862
139,2025-01-17,53.150002,54.490002,52.980000,54.490002,54.490002,54658700,VALE3.SA,1.820004,52.933502,1.0,54.490002,NaN,NaN,NaN,0.000000,1.820004,67.345288,50.869999,55.189999,47.766288,-52.233712,-1.436582,-1.461784,-0.014894,413117562


In [7]:
final_result_df['Ticker'].unique()

array(['ABEV3.SA', 'ITUB4.SA', 'MGLU3.SA', 'PETR4.SA', 'VALE3.SA'],
      dtype=object)

In [8]:
final_result_df.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day_all.xlsx', index=False)

In [9]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(final_result_df.shape[0], final_result_df.shape[1]))

# Any row that has a `NaN` value will be dropped.
final_result_df = final_result_df.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(final_result_df.shape[0], final_result_df.shape[1]))


Antes de tirar as NaN nós tinhamos 140 linhas e 26 colunas
Depois da retirada dos NaN temos 90 linhas e 26 colunas


In [10]:
# Comparar valores e calcular a precisão
final_result_df['Match'] = (final_result_df['Prediction'] == final_result_df['Verify']).astype(int)
accuracy = final_result_df['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 74.44%


#### Uma variavel.

In [6]:
Features = test_day_AMBEV[['RSI','k_percent_14','r_percent_14','Price_Rate_Of_Change','MACD','MACD_EMA','On Balance Volume']]
Features.head()

,RSI,k_percent_14,r_percent_14,Price_Rate_Of_Change,MACD,MACD_EMA,On Balance Volume
4521,54.578924,29.978129,-70.021871,0.120630,0.084563,0.028740,1584192705
4522,52.770921,31.833986,-68.166014,0.122058,0.094006,0.041793,1604251705
4523,53.299328,22.017570,-77.982430,0.123117,0.102294,0.053893,1618896005
4524,53.536070,16.699694,-83.300306,0.124460,0.109657,0.065046,1636932605
4525,52.917572,18.911989,-81.088011,0.124258,0.115659,0.075169,1650984305


In [7]:
Features_array = Features.to_numpy()

# Fazer previsões usando o modelo carregado
predictions = rand_frst_clf_Ambev.predict(Features_array)

# Adicionar as previsões em uma nova coluna chamada 'Prediction'
test_day_AMBEV['Prediction'] = predictions

test_day_AMBEV.tail(30)

c:\ProgramData\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,change_in_price,Smoothed_Close,Prediction,Smoothed_Close_1,Verify,Close_90,Smoothed_Close_90,down_days,up_days,RSI,low_14,high_14,k_percent_14,r_percent_14,MACD,MACD_EMA,Price_Rate_Of_Change,On Balance Volume
5721,2024-12-11,14.27,14.36,13.66,13.80,13.535663,49150600,ABEV3.SA,-0.470000,13.191563,-1.0,13.80,NaN,NaN,NaN,0.470000,0.00,57.824089,12.25,14.53,41.296619,-58.703381,0.078496,0.022718,0.006484,-2731330695
5722,2024-12-12,13.50,13.58,13.10,13.10,12.849072,49616900,ABEV3.SA,-0.700000,13.186069,1.0,13.10,NaN,NaN,NaN,0.700000,0.00,51.759357,12.25,14.53,41.055665,-58.944335,0.092327,0.036640,0.008265,-2780947595
5723,2024-12-13,13.10,13.36,13.01,13.08,12.829453,30867900,ABEV3.SA,-0.020000,13.179705,1.0,13.08,NaN,NaN,NaN,0.020000,0.00,51.600906,12.25,14.53,40.776536,-59.223464,0.101604,0.049633,0.011435,-2811815495
5724,2024-12-16,13.08,13.42,12.84,12.84,12.594051,32095500,ABEV3.SA,-0.240000,13.159323,1.0,12.84,NaN,NaN,NaN,0.240000,0.00,49.729016,12.25,14.53,39.882576,-60.117424,0.106088,0.060924,0.012999,-2843910995
5725,2024-12-17,13.00,13.19,12.99,13.10,12.849072,39564500,ABEV3.SA,0.260000,13.155763,-1.0,13.10,NaN,NaN,NaN,0.000000,0.26,51.675049,12.25,14.53,39.726465,-60.273535,0.108109,0.070361,0.013335,-2883475495
5726,2024-12-18,13.07,13.08,12.75,12.76,12.515584,40864200,ABEV3.SA,-0.340000,13.132018,-1.0,12.76,NaN,NaN,NaN,0.340000,0.00,49.126756,12.25,14.53,38.684983,-61.315017,0.106566,0.077602,0.011472,-2924339695
5727,2024-12-19,12.76,12.91,12.64,12.78,12.535200,38104000,ABEV3.SA,0.020000,13.110896,-1.0,12.78,NaN,NaN,NaN,0.000000,0.02,49.277522,12.60,14.53,26.471307,-73.528693,0.102457,0.082573,0.008415,-2962443695
5728,2024-12-20,12.08,12.30,12.01,12.23,12.230000,86074600,ABEV3.SA,-0.550000,13.058043,1.0,12.23,NaN,NaN,NaN,0.550000,0.00,45.479438,12.01,14.53,41.588992,-58.411008,0.093854,0.084829,0.001998,-3048518295
5729,2024-12-23,12.11,12.34,11.99,12.22,12.220000,40181700,ABEV3.SA,-0.009999,13.007760,-1.0,12.22,NaN,NaN,NaN,0.009999,0.00,45.414229,11.99,14.53,40.069304,-59.930696,0.082034,0.084270,-0.004326,-3088699995
5730,2024-12-26,12.18,12.24,11.99,12.14,12.140000,21166800,ABEV3.SA,-0.080000,12.955694,-1.0,12.14,NaN,NaN,NaN,0.080000,0.00,44.886608,11.99,14.53,38.019478,-61.980522,0.067684,0.080953,-0.010474,-3109866795


In [8]:
test_day_AMBEV.to_excel('C:/Users/Samsung/Documents/GitHub-Datas/TCC_Machine_Learning/Datas referências/test_day.xlsx', index=False)

In [11]:
# We need to remove all rows that have an NaN value.
print('Antes de tirar as NaN nós tinhamos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))

# Any row that has a `NaN` value will be dropped.
test_day_AMBEV = test_day_AMBEV.dropna()

# Display how much we have left now.
print('Depois da retirada dos NaN temos {} linhas e {} colunas'.format(test_day_AMBEV.shape[0], test_day_AMBEV.shape[1]))


Antes de tirar as NaN nós tinhamos 745 linhas e 26 colunas
Depois da retirada dos NaN temos 655 linhas e 26 colunas


In [12]:
# Comparar valores e calcular a precisão
test_day_AMBEV['Match'] = (test_day_AMBEV['Prediction'] == test_day_AMBEV['Verify']).astype(int)
accuracy = test_day_AMBEV['Match'].mean()  # Proporção de acertos (iguais / total)

print(f"\nPrecisão do modelo: {accuracy:.2%}")


Precisão do modelo: 82.90%


<ipython-input-12-80bf6af98153>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_day_AMBEV['Match'] = (test_day_AMBEV['Prediction'] == test_day_AMBEV['Verify']).astype(int)
